In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 26kB/s 
     |████████████████████████████████| 5.9MB 66.0MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7e3d18kr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7e3d18kr
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=b5139ccdaa9e0a3a9da4ec609504ca1b0cedb548ee77364a2fac32072e64abf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-djtzp907/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [8]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-12-22 01:52:37--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-12-22 01:52:37--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb6e2b92f2d34eafbbacadea415.dl.dropboxusercontent.com/cd/0/get/BFgU6yUdoDxmoMbNkqrJ0xIIDDLsEZhdQB4YuDDQKWobv2DAMMrNcs79ox1M1hrmtChtBDFez9-zR6u_8lXushBlxuuM1EwAPEZer6u_etABXYb-3Ch7FE8vZYDmfKc-4A0/file?dl=1# [following]
--2020-12-22 01:52:38--  https://ucb6e2b92f2d34eafbbacadea415.dl.dropboxusercontent.com/cd/0/get/BFgU6yUdoDxmoMbNkqrJ0xIIDDLsEZhdQB4YuDDQKWobv2DAMMrNcs79ox1M1hrmtChtBDFez9-zR6u_8lXushBlxuu

In [9]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [12]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7224432826042175 train acc 0.46875
epoch 1 batch id 201 loss 0.5346298813819885 train acc 0.5515391791044776
epoch 1 batch id 401 loss 0.42719438672065735 train acc 0.6663809226932669
epoch 1 batch id 601 loss 0.482577383518219 train acc 0.7228837354409318
epoch 1 batch id 801 loss 0.3998185396194458 train acc 0.7549937578027466
epoch 1 batch id 1001 loss 0.36366379261016846 train acc 0.7739135864135864
epoch 1 batch id 1201 loss 0.3787617087364197 train acc 0.7882623855120733
epoch 1 batch id 1401 loss 0.3522101044654846 train acc 0.7977449143468951
epoch 1 batch id 1601 loss 0.29116523265838623 train acc 0.8063222204871955
epoch 1 batch id 1801 loss 0.28736525774002075 train acc 0.8133328706274292
epoch 1 batch id 2001 loss 0.3180326521396637 train acc 0.8195667791104447
epoch 1 batch id 2201 loss 0.3240709602832794 train acc 0.8247103589277601

epoch 1 train acc 0.8283560864618885


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8834518861892583


epoch 2 batch id 1 loss 0.5293645858764648 train acc 0.8125
epoch 2 batch id 201 loss 0.19696448743343353 train acc 0.8823849502487562
epoch 2 batch id 401 loss 0.3103269636631012 train acc 0.8855595386533666
epoch 2 batch id 601 loss 0.31021907925605774 train acc 0.8889091098169717
epoch 2 batch id 801 loss 0.3344580829143524 train acc 0.8905079588014981
epoch 2 batch id 1001 loss 0.2709469795227051 train acc 0.8927322677322678
epoch 2 batch id 1201 loss 0.23102441430091858 train acc 0.8949963572023314
epoch 2 batch id 1401 loss 0.2155454158782959 train acc 0.8970155246252677
epoch 2 batch id 1601 loss 0.30625051259994507 train acc 0.8991548251093067
epoch 2 batch id 1801 loss 0.17792069911956787 train acc 0.9010532343142699
epoch 2 batch id 2001 loss 0.20329290628433228 train acc 0.9029547726136932
epoch 2 batch id 2201 loss 0.32035696506500244 train acc 0.9044752385279419

epoch 2 train acc 0.9055167448805461


...


epoch 3 test acc 0.8956601662404092


epoch 4 batch id 1 loss 0.4649967551231384 train acc 0.84375
epoch 4 batch id 201 loss 0.0725564956665039 train acc 0.9581001243781094
epoch 4 batch id 401 loss 0.12187843769788742 train acc 0.9573721945137157
epoch 4 batch id 601 loss 0.21284668147563934 train acc 0.9585326539101497
epoch 4 batch id 801 loss 0.14395275712013245 train acc 0.9601864856429463
epoch 4 batch id 1001 loss 0.07607413828372955 train acc 0.961756993006993
epoch 4 batch id 1201 loss 0.0553576834499836 train acc 0.9623620940882598
epoch 4 batch id 1401 loss 0.030990296974778175 train acc 0.9636085831548894
epoch 4 batch id 1601 loss 0.13537707924842834 train acc 0.9643874921923797
epoch 4 batch id 1801 loss 0.023324251174926758 train acc 0.9655313020544142
epoch 4 batch id 2001 loss 0.039879921823740005 train acc 0.9664308470764618
epoch 4 batch id 2201 loss 0.19517487287521362 train acc 0.9668829509313949

epoch 4 train acc 0.9674190308589307



epoch 4 test acc 0.8970388427109974


epoch 5 batch id 1 loss 0.32698675990104675 train acc 0.859375
epoch 5 batch id 201 loss 0.018552079796791077 train acc 0.9731032338308457
epoch 5 batch id 401 loss 0.047522976994514465 train acc 0.9742051122194514
epoch 5 batch id 601 loss 0.157746821641922 train acc 0.9754575707154742
epoch 5 batch id 801 loss 0.06049860268831253 train acc 0.9759675405742821
epoch 5 batch id 1001 loss 0.005698040593415499 train acc 0.9770854145854145
epoch 5 batch id 1201 loss 0.024382341653108597 train acc 0.9776748542880933
epoch 5 batch id 1401 loss 0.03602411970496178 train acc 0.9782075303354747
epoch 5 batch id 1601 loss 0.07366960495710373 train acc 0.9785973610243598
epoch 5 batch id 1801 loss 0.014344255439937115 train acc 0.9791175041643532
epoch 5 batch id 2001 loss 0.012487486004829407 train acc 0.9794321589205397
epoch 5 batch id 2201 loss 0.0838836133480072 train acc 0.979462460245343

epoch 5 train acc 0.9796688353242321



epoch 5 test acc 0.8975183823529411
